In [2]:
import pandas as pd
import requests
import datetime
import plotly.graph_objects as go
from tqdm import tqdm
from transformers import pipeline, AutoTokenizer
import warnings
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import re
import time
import json
import numpy as np
from bs4 import BeautifulSoup
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from bs4 import BeautifulSoup


warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
<ipython-input-2-65bc2096cd24>:17: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-2-65bc2096cd24>:18: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


## Fetching Demography Data from Census.gov

In [3]:
"""
  Getting 2 letter code of all the states in US from wikipedia by scraping
  https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States
"""

def get_state_codes():

  url = 'https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States'
  response = requests.get(url)
  html_content = response.text

  soup = BeautifulSoup(html_content, 'html.parser')
  table = soup.find('table', {'class': 'wikitable'})
  data = []
  for row in table.find_all('tr')[1:]:  # Skipping the header row
      header_cell = row.find('th')
      first_data_cell = row.find('td')

      if header_cell and first_data_cell:
          state_link = header_cell.find('a')
          if state_link:
              state_name = state_link.text.strip()
              state_name = state_name.replace(' ', '')
          abbreviation = first_data_cell.text.strip()
          data.append([state_name, abbreviation])

  state_codes = pd.DataFrame(data, columns=['State', 'Abbreviation'])

  return state_codes

In [4]:
state_codes = get_state_codes()

In [5]:
"""
  Getting Federal Information Processing Standard (FIPS) code from census.gov website
  https://www.census.gov/library/reference/code-lists/ansi/ansi-codes-for-states.html
"""
def get_fips_code():

  url = "https://www.census.gov/library/reference/code-lists/ansi/ansi-codes-for-states.html"
  response = requests.get(url)
  soup = BeautifulSoup(response.content, 'html.parser')
  rows = soup.find_all('table')[0].find_all('tr')[1:]  # Assuming the first table is the one we need

  state_fips_codes = {}

  for row in rows:
      cols = row.find_all('td')
      state_name = cols[0].text.strip()
      state_name = state_name.replace(' ', '')
      fips_code = cols[1].text.strip()
      state_fips_codes[state_name] = fips_code

  state_fips_codes = pd.DataFrame(list(state_fips_codes.items()), columns=['State', 'FIPS Code'])

  return state_fips_codes

In [6]:
state_fips_codes = get_fips_code()

In [7]:
## Merging state codes and fips code tables
data = state_codes.merge(state_fips_codes[['State', 'FIPS Code']], on='State', how='left')

In [8]:
## Manually map few States to their required name as per their subreddit name
manual_mappings = {'Arizona': 'arizona', 'Alaska': 'alaska', 'Florida': 'florida',
                   'Illinois': 'illinois', 'Kansas': 'kansas', 'Maryland': 'maryland',
                   'Massachusetts': 'massachusetts', 'Minnesota': 'minnesota',
                   'Mississippi': 'mississippi', 'Missouri': 'missouri',
                   'Oklahoma': 'oklohoma', 'Oregon': 'oregon',
                   'Vermont': 'vermont', 'Wisconsin': 'wisconsin', 'Wyoming': 'wyoming',
                   'NewHampshire': 'newhampshire', 'NewJersey': 'newjersey', 'NewYork': 'newyork',
                   'NorthDakota': 'northdakota', 'SouthCarolina': 'southcarolina', 'oklohoma': 'Oklahoma'}

data['State'] = data['State'].apply(lambda x: manual_mappings[x] if x in manual_mappings else x)

In [9]:
data

,State,Abbreviation,FIPS Code
0,Alabama,AL,01
1,alaska,AK,02
2,arizona,AZ,04
3,Arkansas,AR,05
4,California,CA,06
5,Colorado,CO,08
6,Connecticut,CT,09
7,Delaware,DE,10
8,florida,FL,12
9,Georgia,GA,13


In [10]:
'''
  This function fetches demography data of particular state given as fips_code from given set of tables
  https://api.census.gov/data/2019/acs/acs5/variables.json

  fips_code : FIPS State Code
  get : Table name identified from (https://api.census.gov/data/2019/acs/acs1/variables.json)
  key : Fetch data for 'key' demography
'''

def fetch_data(fips_code, tables, key=None):
  base_url = "https://api.census.gov/data/2019/acs/acs5"
  params = {
      'get': tables,
      'for': f'state:{fips_code}'
  }
  response = requests.get(base_url, params=params)
  time.sleep(1)

  if response.status_code == 200:
      data = response.json()
      if(key == 'income'):
        return int(data[1][0])
      elif(key == 'young_population'):
        return sum(map(int, data[1][1:-1]))
      else:
        return sum(map(int, data[1][:-1]))
  else:
      return None

In [11]:
def plot(state_codes, data, title, colorbar_title):
  fig = go.Figure(data=go.Choropleth(
      locations=state_codes,
      z=data.astype(float),
      locationmode='USA-states',
      colorscale='Greens',
      colorbar_title=colorbar_title,
  ))

  fig.update_layout(
      title_text=title,
      geo_scope='usa',
  )

  fig.show()

In [12]:
## B01001_006E-B01001_011E - 15 years to 29 years males
## B01001_030E-B01001_035E - 15 years to 29 years females
data['young_population'] = data['FIPS Code'].apply(lambda x: fetch_data(x, 'B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,B01001_030E,B01001_031E,B01001_032E,B01001_033E,B01001_034E,B01001_035E', key='young_population'))
## B19013_001E - Estimate median household income
data['income'] = data['FIPS Code'].apply(lambda x: fetch_data(x, 'B19013_001E', key='income'))
## B15003_022E - people with bachelor's degree
## B15003_023E - people with master's degree
## B15003_024E - people with other professional degree
## B15003_025E - people with PHD degree
data['education'] = data['FIPS Code'].apply(lambda x: fetch_data(x, 'B15003_022E,B15003_023E,B15003_024E,B15003_025E'))
## B01001_011E - 25 years to
data['adult_population'] = data['FIPS Code'].apply(lambda x: fetch_data(x, 'B01001_012E,B01001_013E,B01001_014E,B01001_015E,B01001_016E,B01001_017E,B01001_018E,B01001_019E,B01001_036E,B01001_037E,B01001_038E,B01001_039E,B01001_040E,B01001_041E,B01001_042E,B01001_043E'))
## B01001_002E - number of male population
data['male_population'] = data['FIPS Code'].apply(lambda x: fetch_data(x, 'B01001_002E'))
## B01001_026E - number of female population
data['female_population'] = data['FIPS Code'].apply(lambda x: fetch_data(x, 'B01001_026E'))

In [17]:
data

,State,Abbreviation,FIPS Code,young_population,income,education,male_population,female_population,adult_population
0,Alabama,AL,01,885031,50536,845772,2359355,2516895,2183374
1,alaska,AK,02,148171,77640,142019,384915,352153,335798
2,arizona,AZ,04,1320500,58945,1394526,3504509,3545790,3024423
3,Arkansas,AR,05,543996,47597,463236,1471760,1527610,1310861
4,California,CA,06,7615269,75235,8980726,19526298,19757199,17907010
5,Colorado,CO,08,1066669,72331,1565134,2823201,2787148,2612811
6,Connecticut,CT,09,637965,78444,975465,1744245,1830829,1660939
7,Delaware,DE,10,167750,68287,214138,462890,494358,428729
8,florida,FL,12,3504562,55660,4471701,10220813,10680823,9363521
9,Georgia,GA,13,1967121,58700,2157616,5062096,5341751,4738190


In [ ]:
data.to_csv('demography_data.csv', index=False)
demography = pd.read_csv('demography_data.csv')

## Scraping state wise reddit comments from r/state_name

In [ ]:
states = list(data['State'].values) ## total states in united states
topics = ['economy', 'COVID', 'BLM', 'Healthcare', 'immigrant', 'lgbtq', 'abortion'] ## Major topics that could affect US elections
model_name = 'cardiffnlp/twitter-roberta-base-sentiment' ## Sentiment analysis model trained on twitter comments
sentiment_pipeline = pipeline('sentiment-analysis',  model=model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [ ]:
def clean_comment(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove user mentions
    text = re.sub(r'@\w+', '', text)

    # Remove hashtags 
    text = re.sub(r'#', '', text)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove special characters
    text = re.sub(r'[^\w\s]', '', text)

    # Remove numbers
    text = re.sub(r'\d', '', text)

    return text


In [ ]:
## Using pullpush API to scrape comments from subreddit based on given topic (query) and time period
def getPullPushData(query, after, before, sub):
    url = 'https://api.pullpush.io/reddit/search/comment/?q='+str(query)+'&size=1000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    r = requests.get(url)
    data = json.loads(r.text)
    time.sleep(1)
    return data['data']

In [ ]:
def collect_and_analyze_data(state, topic, comments):
    initial = len(comments)
    ## before and after are selected according to Trump's Tenure
    after = "1478592000"  # Nov 08, 2016
    before = "1604390400"  # Nov 03, 2020
    data = getPullPushData(topic, after, before, state)

    while len(data) > 0:
        for submission in data:
            try:
                comment = submission['body']
                comment = clean_comment(comment) ## Cleaning data

                if 'score' in submission:
                  ups = abs(submission['score'])
                else:
                  ups = 1

                ## generating tokens from comments
                tokens = tokenizer.encode(comment, add_special_tokens=True, truncation=True, max_length=1024)
                if len(tokens) > 512:
                    try:
                        ## Generating summary to limit the comment length for sentiment analysis model
                        summary_result = summarizer(comment, max_length=200, min_length=30, do_sample=False)
                        if summary_result and 'summary_text' in summary_result[0]:
                            summary = summary_result[0]['summary_text']
                        else:
                            print("No summary was returned for a comment, using original comment as summary.")
                            summary = comment
                    except Exception as e:
                        print(f"Error in summarization: {e}")
                        summary = comment
                else:
                    summary = comment

                sentiment_result = sentiment_pipeline(summary)[0]
                if(sentiment_result['label'] == 'LABEL_0'): ## LABEL_0 is negative sentiment
                    sentiment_score = -sentiment_result['score']
                else: ## LABEL_1 and LABEL_2 is positive and neutral sentiment
                    sentiment_score = sentiment_result['score']

                comments.append({
                    'state': state,
                    'topic': topic,
                    'comment': comment,
                    'sentiment_score': sentiment_score,
                    'score': ups
                })
            except Exception as e:
                print(f"An error occurred during processing: {e}")

        if data:
            before = data[-1]['created_utc']
            data = getPullPushData(topic, after, before, state)

    final = len(comments)
    print(f"Total comments for {state}: ", final - initial)

    return comments

In [ ]:
## This function weighs the comments according to the score or upvotes given to comments
def weighted_avg_and_count(group):
    if group['score'].sum() == 0:
        weighted_avg = group['sentiment_score'].mean()
    else:
        weighted_avg = np.average(group['sentiment_score'], weights=group['score'])
    count = group['comment'].count()
    return pd.Series([weighted_avg, count], index=['weighted_sentiment_score', 'comment_count'])


In [ ]:
for topic in topics:
  comments = []
  for state in tqdm(states):
      comments = collect_and_analyze_data(state, topic, comments)
  comments_df = pd.DataFrame(comments)
  comments_df.to_csv(f'comments_{topic}.csv')
  state_sentiments = comments_df.groupby('state').apply(weighted_avg_and_count).reset_index()
  topic = topic.replace(' ', '_')
  state_sentiments.to_csv(f'{topic}.csv', index=False)

In [ ]:
comments_table = pd.read_csv("comments_abortion.csv", index_col=0)
comments_table.head()

,state,topic,comment,sentiment_score,score
0,Alabama,abortion,No one can legally force you to get an abortio...,-0.848300,6.0
1,Alabama,abortion,Abortions are not illegal in Alabama,-0.587977,2.0
2,Alabama,abortion,He cannot legally make you have an abortion,-0.736194,7.0
3,Alabama,abortion,gtCan he legally remove me without formal noti...,-0.934370,7.0
4,Alabama,abortion,Must give credit where credit is due Trump has...,-0.608584,6.0


In [32]:
def plot_sentiments(data, topic):
  fig = go.Figure(data=go.Choropleth(
      locations=data['Abbreviation'],
      z=data['weighted_sentiment_score'].astype(float),
      locationmode='USA-states',
      colorscale='RdBu',
      colorbar_title="Sentiment Score",
      zmin=-1,
      zmax=1,
  ))

  fig.update_layout(
      title_text=f'State-wise Sentiment on {topic}',
      geo_scope='usa',
  )

  fig.show()

plot_sentiments(data, 'economy')

In [ ]:
## loading sentiment scores of all the topics generated above
abortion = pd.read_csv('state_sentiments_abortion.csv')
blm = pd.read_csv('state_sentiments_blm.csv')
covid = pd.read_csv('state_sentiments_covid.csv')
economy = pd.read_csv('state_sentiments_economy.csv')
healthcare = pd.read_csv('state_sentiments_healthcare.csv')
immigrant = pd.read_csv('state_sentiments_immigrant.csv')
lgbtq = pd.read_csv('state_sentiments_lgbtq.csv')

In [ ]:
abortion.rename(columns={'weighted_sentiment_score': 'abortion_sentiment'}, inplace=True)
blm.rename(columns={'weighted_sentiment_score': 'blm_sentiment'}, inplace=True)
covid.rename(columns={'weighted_sentiment_score': 'covid_sentiment'}, inplace=True)
economy.rename(columns={'weighted_sentiment_score': 'economy_sentiment'}, inplace=True)
healthcare.rename(columns={'weighted_sentiment_score': 'healthcare_sentiment'}, inplace=True)
immigrant.rename(columns={'weighted_sentiment_score': 'immigrant_sentiment'}, inplace=True)
lgbtq.rename(columns={'weighted_sentiment_score': 'lgbtq_sentiment'}, inplace=True)

In [ ]:
## Merging all topics
combined_df = pd.merge(abortion[['state', 'abortion_sentiment']],
                       blm[['state', 'blm_sentiment']],
                       on='state', how='outer')

combined_df = pd.merge(combined_df, covid[['state', 'covid_sentiment']], on='state', how='outer')
combined_df = pd.merge(combined_df, economy[['state', 'economy_sentiment']], on='state', how='outer')
combined_df = pd.merge(combined_df, healthcare[['state', 'healthcare_sentiment']], on='state', how='outer')
combined_df = pd.merge(combined_df, immigrant[['state', 'immigrant_sentiment']], on='state', how='outer')
combined_df = pd.merge(combined_df, lgbtq[['state', 'lgbtq_sentiment']], on='state', how='outer')

In [ ]:
combined_df

,state,abortion_sentiment,blm_sentiment,covid_sentiment,economy_sentiment,healthcare_sentiment,immigrant_sentiment,lgbtq_sentiment
0,Alabama,-0.376950,-0.195277,-0.286856,-0.325519,-0.361859,-0.387746,-0.193995
1,Arkansas,-0.437251,-0.226826,-0.269840,-0.412520,-0.333640,-0.152638,-0.158848
2,California,-0.326307,0.002632,-0.050473,0.060448,-0.150958,-0.237524,0.022265
3,Colorado,-0.309837,-0.116862,-0.147337,-0.241285,-0.344046,-0.292442,0.113118
4,Connecticut,-0.484616,-0.475845,-0.151420,-0.206024,-0.182960,-0.333684,-0.388699
5,Delaware,-0.471206,-0.357474,-0.282107,-0.255518,-0.392421,-0.402703,0.142113
6,Georgia,-0.512242,-0.294188,-0.285018,-0.317941,-0.311354,-0.558837,-0.234335
7,Hawaii,-0.313205,-0.393398,-0.236998,-0.219227,-0.208349,-0.223650,0.072662
8,Idaho,-0.432274,-0.326231,-0.359597,-0.367301,-0.339245,-0.412786,-0.566649
9,Indiana,-0.500831,-0.531928,-0.329805,-0.403014,-0.405179,-0.434884,-0.385232


In [ ]:
combined_df.to_csv('sentiment_scores.csv')

In [23]:
sentiment_scores = pd.read_csv('sentiment_scores.csv', index_col=0)
demography_data = pd.read_csv('demography_data.csv', index_col=0)

In [ ]:
demography_data

,State,Abbreviation,FIPS Code,young_population,income,education,adult_population,male_population,female_population
0,Alabama,AL,1,885031,50536,845772,1218746,2359355,2516895.0
1,alaska,AK,2,148171,77640,142019,207790,384915,352153.0
2,arizona,AZ,4,1320500,58945,1394526,1760691,3504509,3545790.0
3,Arkansas,AR,5,543996,47597,463236,744414,1471760,1527610.0
4,California,CA,6,7615269,75235,8980726,10527413,19526298,19757199.0
5,Colorado,CO,8,1066669,72331,1565134,1550686,2823201,2787148.0
6,Connecticut,CT,9,637965,78444,975465,923931,1744245,1830829.0
7,Delaware,DE,10,167750,68287,214138,239635,462890,494358.0
8,florida,FL,12,3504562,55660,4471701,5288301,10220813,10680823.0
9,Georgia,GA,13,1967121,58700,2157616,2658195,5062096,5341751.0


In [ ]:
## Add Abbreviation column to merged df
state_to_abbreviation = demography_data.set_index('State')['Abbreviation'].to_dict()
state_to_abbreviation = {key.lower(): value for key, value in state_to_abbreviation.items()}
sentiment_scores['state'] = sentiment_scores['state'].apply(lambda x: x.lower())
sentiment_scores['Abbreviation'] = sentiment_scores['state'].map(state_to_abbreviation)
sentiment_scores.to_csv("sentiment_scores.csv")
sentiment_scores = pd.read_csv("sentiment_scores.csv", index_col=0)

In [ ]:
sentiment_scores

,state,abortion_sentiment,blm_sentiment,covid_sentiment,economy_sentiment,healthcare_sentiment,immigrant_sentiment,lgbtq_sentiment,Abbreviation
0,alabama,-0.376950,-0.195277,-0.286856,-0.325519,-0.361859,-0.387746,-0.193995,AL
1,arkansas,-0.437251,-0.226826,-0.269840,-0.412520,-0.333640,-0.152638,-0.158848,AR
2,california,-0.326307,0.002632,-0.050473,0.060448,-0.150958,-0.237524,0.022265,CA
3,colorado,-0.309837,-0.116862,-0.147337,-0.241285,-0.344046,-0.292442,0.113118,CO
4,connecticut,-0.484616,-0.475845,-0.151420,-0.206024,-0.182960,-0.333684,-0.388699,CT
5,delaware,-0.471206,-0.357474,-0.282107,-0.255518,-0.392421,-0.402703,0.142113,DE
6,georgia,-0.512242,-0.294188,-0.285018,-0.317941,-0.311354,-0.558837,-0.234335,GA
7,hawaii,-0.313205,-0.393398,-0.236998,-0.219227,-0.208349,-0.223650,0.072662,HI
8,idaho,-0.432274,-0.326231,-0.359597,-0.367301,-0.339245,-0.412786,-0.566649,ID
9,indiana,-0.500831,-0.531928,-0.329805,-0.403014,-0.405179,-0.434884,-0.385232,IN


In [ ]:
sentiment_scores.to_csv('sentiment_scores.csv')

In [ ]:
demography_data

,Abbreviation,FIPS Code,young_population,income,education,adult_population,male_population,female_population
0,AK,2,148171,77640,142019,207790,384915,352153.0
1,AL,1,885031,50536,845772,1218746,2359355,2516895.0
2,AR,5,543996,47597,463236,744414,1471760,1527610.0
3,AZ,4,1320500,58945,1394526,1760691,3504509,3545790.0
4,CA,6,7615269,75235,8980726,10527413,19526298,19757199.0
5,CO,8,1066669,72331,1565134,1550686,2823201,2787148.0
6,CT,9,637965,78444,975465,923931,1744245,1830829.0
7,DE,10,167750,68287,214138,239635,462890,494358.0
8,FL,12,3504562,55660,4471701,5288301,10220813,10680823.0
9,GA,13,1967121,58700,2157616,2658195,5062096,5341751.0


In [ ]:
"""

  Access our app on thish site : https://data-and-web-technologies-for-data.onrender.com/

"""

demography_data = pd.read_csv('demography_data.csv', index_col=0)
sentiment_scores = pd.read_csv('sentiment_scores.csv', index_col=0)

demography_data = demography_data.sort_values(by='Abbreviation').reset_index(drop=True)
sentiment_scores = sentiment_scores.sort_values(by='Abbreviation').reset_index(drop=True)

app = dash.Dash(__name__)

app.layout = html.Div(style={'backgroundColor': '#f5f5f5'},  # Common background color
                      children=[
    html.H1('State wise correlation between various national topics and their respective sentiment scores', style={'textAlign': 'center', 'color': '#333'}),
    html.Div([
        html.Div([
            dcc.Dropdown(
                id='demographic-dropdown',
                options=[{'label': col, 'value': col} for col in demography_data.columns if col not in ['State', 'Abbreviation', 'FIPS Code']],
                value='income'
            ),
        ], style={'width': '45%', 'display': 'inline-block', 'padding': '10px'}),

        html.Div([
            dcc.Dropdown(
                id='topic-dropdown',
                options=[{'label': col, 'value': col} for col in sentiment_scores.columns if col not in ['state', 'Abbreviation']],
                value='abortion_sentiment'
            ),
        ], style={'width': '45%', 'float': 'right', 'display': 'inline-block', 'padding': '10px'}),
    ]),
    html.Div([
        dcc.Graph(id='demographic-map', style={'height': '50vh'}),
    ], style={'width': '45%', 'display': 'inline-block', 'padding': '10px'}),

    html.Div([
        dcc.Graph(id='sentiment-map', style={'height': '50vh'}),
    ], style={'width': '50%', 'float': 'right', 'display': 'inline-block', 'padding': '10px'}),

    html.Div([
        dcc.Graph(id='scatter-plot', style={'height': '60vh'}),
    ], style={'width': '90%', 'display': 'block', 'marginLeft': 'auto', 'marginRight': 'auto', 'padding': '10px 20px'}),

])

@app.callback(
    Output('demographic-map', 'figure'),
    Input('demographic-dropdown', 'value')
)
def update_demographic_map(selected_demographic):
    fig = px.choropleth(
        demography_data,
        locations='Abbreviation',
        locationmode="USA-states",
        color=selected_demographic,
        scope="usa",
        color_continuous_scale='Blues'
    )
    return fig

@app.callback(
    Output('sentiment-map', 'figure'),
    Input('topic-dropdown', 'value')
)
def update_sentiment_map(selected_topic):
    fig = px.choropleth(
        sentiment_scores,
        locations='Abbreviation',
        locationmode="USA-states",
        color=selected_topic,
        scope="usa",
        color_continuous_scale='RdBu',
        range_color=[-1, 1]
    )
    return fig

@app.callback(
    Output('scatter-plot', 'figure'),
    [Input('demographic-dropdown', 'value'),
     Input('topic-dropdown', 'value')]
)
def update_scatter_plot(selected_demographic, selected_topic):
    X = np.array(demography_data[selected_demographic].values).reshape(-1, 1)
    y = np.array(sentiment_scores[selected_topic].values)

    reg = LinearRegression().fit(X, y)

    line_X = np.linspace(X.min(), X.max(), 100).reshape(-1, 1)
    line_y = reg.predict(line_X)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=X.ravel(), y=y, mode='markers', name='Data'))
    fig.add_trace(go.Scatter(x=line_X.ravel(), y=line_y, mode='lines', name='Regression Line'))

    fig.update_xaxes(title_text=f'{selected_demographic}')
    fig.update_yaxes(title_text=f'{selected_topic}')

    return fig

server = app.server

if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>